In [ ]:
import os
import configparser
import cv2
import numpy as np
from pero_ocr.core.layout import PageLayout
from pero_ocr.document_ocr.page_parser import PageParser
import re
from ultralytics import YOLO
import pandas as pd


In [ ]:
def predict(chosen_model, img, classes=[], conf=0.5):
    if classes:
        results = chosen_model.predict(img, classes=classes, conf=conf)
    else:
        results = chosen_model.predict(img, conf=conf)

    return results


def predict_and_detect(chosen_model, img, classes=[], conf=0.5, rectangle_thickness=5, text_thickness=1):
    results = predict(chosen_model, img, classes, conf=conf)
    for result in results:
        for box in result.boxes:
            cv2.rectangle(img, (int(box.xyxy[0][0]), int(box.xyxy[0][1])),
                          (int(box.xyxy[0][2]), int(box.xyxy[0][3])), (255, 0, 255), rectangle_thickness)
            # cv2.putText(img, f"{result.names[int(box.cls[0])]}",
            #             (int(box.xyxy[0][0]), int(box.xyxy[0][1]) - 10),
            #             cv2.FONT_HERSHEY_PLAIN, 1, (255, 0, 0), text_thickness)
    return img, results


            

In [ ]:
model = YOLO("best.pt")
path=os.getcwd()+"\\images_a_traiter"
for root, dirs, file in os.walk(path):
    for f in file:
        if ".jpg" in f:
    # read the image
            image = cv2.imread(f"images_a_traiter\\"+f)
            result_img, results = predict_and_detect(model, image, classes=[], conf=0.5)
            for result in results:
                    df=result.to_df()
                    df.to_csv(f"{f}_annots.csv")
            # Decommenter ce qui suit pour obtenir également les coordonnées dans un fichier texte.
            # with open(f"coord_{f}.txt",'a') as file:
              
                    # for box in result.boxes:
                    #      file.write(str(box)+"\n")
        #Decommenter ce qui suit pour afficher une visualisation de l'image qui affiche les zones extraites.
        # cv2.imshow("Image", result_img)
        # cv2.waitKey(0)
            cv2.imwrite(f"result{f}.png", result_img)
            

In [ ]:



#Changer le fichier de config vers config_cpu.ini si l'on ne dispose pas d'un GPU.
config_path = "pero_eu_cz_print_newspapers_2022-09-26\\config.ini"
config = configparser.ConfigParser()
config.read(config_path)

page_parser = PageParser(config, config_path=os.path.dirname(config_path))


# Read the document page image.
path=os.getcwd()+"\\images_a_traiter"
os.makedirs("dossier_output_experience", exist_ok=True)
for f in os.listdir(path):
        
            
            input_image_path = f"images_a_traiter\\{f}"
            image = cv2.imread(input_image_path, 1)

            # Init empty page content. 
            # This object will be updated by the ocr pipeline. id can be any string and it is used to identify the page.
            page_layout = PageLayout(id=input_image_path, page_size=(image.shape[0], image.shape[1]))

        # Process the image by the OCR pipeline
        # try:
            page_layout = page_parser.process_page(image, page_layout)


            page_layout.to_pagexml(f'dossier_output_experience\\output_page{f}.xml') # Save results as Page XML.
            page_layout.to_altoxml(f'dossier_output_experience\\output_ALTO{f}.xml') # Save results as ALTO XML.
            for region in page_layout.regions:
                
                liste_lignes = [f"Zone_texte",]

                for line in region.lines:
                    liste_lignes.append(line.transcription)

                with open(f'dossier_output_experience\\texte{f}.txt', 'a', encoding='utf-8') as txt:
                    txt.write('\n')
                    for element in liste_lignes:
                        txt.write(element +" ")
        # except:
        #     print(f"probleme avec {f}")
        #     pass

        # Render detected text regions and text lines into the image and
        # save it into a file.
            rendered_image = page_layout.render_to_image(image) 
            cv2.imwrite(f'results_full2\\{f}.png', rendered_image)

            # Save each cropped text line in a separate .jpg file.
            # for region in page_layout.regions:
            #     for line in region.lines:
            #         cv2.imwrite(f'file_id-{line.id}.jpg', line.crop.astype(np.uint8))
        